# Simple Evaluation Function

This notebook can return a static centipawn value for each chess position. It will not look into the future and only consider the current state of the board. 

This method has been developed by Tomasz Michniewski in the Polish chess programming discussion list (progszach).


https://www.chessprogramming.org/Simplified_Evaluation_Function

In [1]:
using Pkg
# Pkg.add("Chess")
using Chess

In [2]:
pawn_square_table = [
     0,  0,  0,  0,  0,  0,  0,  0,
     5, 10, 10,-20,-20, 10, 10,  5,
     5, -5,-10,  0,  0,-10, -5,  5,
     0,  0,  0, 20, 20,  0,  0,  0,
     5,  5, 10, 25, 25, 10,  5,  5,
    10, 10, 20, 30, 30, 20, 10, 10,
    50, 50, 50, 50, 50, 50, 50, 50,
     0,  0,  0,  0,  0,  0,  0,  0
]


knight_square_table = [
    -50,-40,-30,-30,-30,-30,-40,-50,
    -40,-20,  0,  5,  5,  0,-20,-40,
    -30,  5, 10, 15, 15, 10,  5,-30,
    -30,  0, 15, 20, 20, 15,  0,-30,
    -30,  5, 15, 20, 20, 15,  5,-30,
    -30,  0, 10, 15, 15, 10,  0,-30,
    -40,-20,  0,  0,  0,  0,-20,-40,
    -50,-40,-30,-30,-30,-30,-40,-50
]

bishop_square_table = [
    -20,-10,-10,-10,-10,-10,-10,-20,
    -10,  5,  0,  0,  0,  0,  5,-10,
    -10, 10, 10, 10, 10, 10, 10,-10,
    -10,  0, 10, 10, 10, 10,  0,-10,
    -10,  5,  5, 10, 10,  5,  5,-10,
    -10,  0,  5, 10, 10,  5,  0,-10,
    -10,  0,  0,  0,  0,  0,  0,-10,
    -20,-10,-10,-10,-10,-10,-10,-20
]

rook_square_table = [
     0,  0,  0,  5,  5,  0,  0,  0,
    -5,  0,  0,  0,  0,  0,  0, -5,
    -5,  0,  0,  0,  0,  0,  0, -5,
    -5,  0,  0,  0,  0,  0,  0, -5,
    -5,  0,  0,  0,  0,  0,  0, -5,
    -5,  0,  0,  0,  0,  0,  0, -5,
     5, 10, 10, 10, 10, 10, 10,  5,
     0,  0,  0,  0,  0,  0,  0,  0
]

queen_square_table = [
    -20,-10,-10, -5, -5,-10,-10,-20,
    -10,  0,  0,  0,  0,  0,  0,-10,
    -10,  5,  5,  5,  5,  5,  0,-10,
     0,  0,  5,  5,  5,  5,  0, -5,
     0,  5,  5,  5,  5,  5,  0, -5,
    -10,  0,  5,  5,  5,  5,  0,-10,
    -10,  0,  0,  0,  0,  0,  0,-10,
    -20,-10,-10, -5, -5,-10,-10,-20
]

king_square_middle_game_table = [
    20, 30, 10,  0,  0, 10, 30, 20,
    20, 20,  0,  0,  0,  0, 20, 20,
    -10,-20,-20,-20,-20,-20,-20,-10,
    -20,-30,-30,-40,-40,-30,-30,-20,
    -30,-40,-40,-50,-50,-40,-40,-30,
    -30,-40,-40,-50,-50,-40,-40,-30,
    -30,-40,-40,-50,-50,-40,-40,-30,
    -30,-40,-40,-50,-50,-40,-40,-40
]

king_square_end_game_table = [
    -50,-40,-30,-20,-20,-30,-40,-50,
    -30,-20,-10,  0,  0,-10,-20,-30,
    -30,-10, 20, 30, 30, 20,-10,-30,
    -30,-10, 30, 40, 40, 30,-10,-30,
    -30,-10, 30, 40, 40, 30,-10,-30,
    -30,-10, 20, 30, 30, 20,-10,-30,
    -30,-30,  0,  0,  0,  0,-30,-30,
    -50,-30,-30,-30,-30,-30,-30,-50
]

64-element Vector{Int64}:
 -50
 -40
 -30
 -20
 -20
 -30
 -40
 -50
 -30
 -20
 -10
   0
   0
   ⋮
   0
   0
 -30
 -30
 -50
 -30
 -30
 -30
 -30
 -30
 -30
 -50

`get_files()` returns a tuple with all file names as chars.

In [3]:
function get_files()
    return ('a', 'b', 'c', 'd', 'e', 'f', 'g', 'h')
end

get_files (generic function with 1 method)

In [4]:
get_files()

('a', 'b', 'c', 'd', 'e', 'f', 'g', 'h')

`get_Index_by_File` returns the number according to the file char. 'a' => 1, 'b' => 2, ...

In [5]:
function get_Index_by_File(myfile::Char)
    files=get_files()
    return findfirst(item -> item == myfile, files)
end

get_Index_by_File (generic function with 1 method)

In [6]:
get_Index_by_File('d')

4

More efficient solution for get_Intex_by_file

In [7]:
Int('a') - 96

1

`get_piece` returns the Piece on a specific square of a board.

In [8]:
function get_piece(board, square)
    return pieceon(board, square)
end

get_piece (generic function with 1 method)

In [9]:
function get_piece(board, file, rank)
    square = squarefromstring(file * string(rank))
    return pieceon(board, square)
end

get_piece (generic function with 2 methods)

In [10]:
piece = get_piece(startboard(), "g", "3")
if(piece == EMPTY)
    print("keine Figur vorhanden")
end
ptype(piece)

keine Figur vorhanden

PIECE_TYPE_NONE

`get_value` returns the centipawn value of a chess Piece according to the article linked at the top of the notebook.

In [11]:
function get_value(piece::Piece)
    
    if ptype(piece) == PAWN
        return 100
    elseif ptype(piece) == KNIGHT
        return 320
    elseif ptype(piece) == BISHOP
        return 330
    elseif ptype(piece) == ROOK
        return 500
    elseif ptype(piece) == QUEEN
        return 900
    elseif ptype(piece) == KING 
        return 20000
    else
    print("Figur nicht vorhanden")
    return nothing        

    end 
end

get_value (generic function with 1 method)

In [12]:
piece = get_piece(startboard(), "g", "1")
get_value(piece)

320

Additionally we should define where the ending begins. For me it might be either if:

Both sides have no queens or
Every side which has a queen has additionally no other pieces or one minorpiece maximum.

In [13]:
function is_endgame(board)
    # Check if both sides have no queens
    if(isempty(queens(board)))
        return true
    end 
     # Check if each side that has a queen also has no more than one minor piece
    num_white_minor_pieces = squarecount(knights(board, WHITE)) + squarecount(bishops(board, WHITE))
    num_black_minor_pieces = squarecount(knights(board, BLACK)) + squarecount(bishops(board, BLACK))
    if(isempty(rooks(board)) && num_white_minor_pieces <= 1 && num_black_minor_pieces <= 1 )
        return true
    end
    return false
end

is_endgame (generic function with 1 method)

In [14]:
is_endgame(startboard())

false

In [42]:
is_endgame(fromfen("r3k2r/pppppppp/8/8/8/8/PPPPPPPP/R3K2R w KQkq - 0 1"))

true

`get_square_value`...

In [16]:
function get_square_value(board::Board, piece::Piece, x, y)
    # Bestimmen Sie den Wert des Feldes für diese Schachfigur
    if ptype(piece) == PAWN
        square_table = pawn_square_table
    elseif ptype(piece) == KNIGHT
        square_table = knight_square_table
    elseif ptype(piece) == BISHOP
        square_table = bishop_square_table
    elseif ptype(piece) == ROOK
        square_table = rook_square_table
    elseif ptype(piece) == QUEEN
        square_table = queen_square_table
    elseif ptype(piece) == KING
        if(is_endgame(board))
            square_table = king_square_end_game_table
        else
            square_table = king_square_middle_game_table
        end
    else
        print("Figur nicht vorhanden")
        return nothing
    end

    if Chess.pcolor(piece) == WHITE
        return square_table[x + 8*(y-1)]
   elseif Chess.pcolor(piece) == BLACK
        square_table = reverse(square_table)
        return square_table[x + 8*(y-1)]
    end
end

get_square_value (generic function with 1 method)

In [17]:
function calc_score(board, piece, score, x, y)
    if Chess.pcolor(piece) == WHITE
        score += get_value(piece) + get_square_value(board, piece, x, y)
    elseif Chess.pcolor(piece) == BLACK
        score -= get_value(piece) + get_square_value(board, piece, x, y)
    end
    return score
end

calc_score (generic function with 1 method)

In [18]:
function calc_score(board, piece, score, square)
    myfile = get_Index_by_File(tochar(file(square)))
    myrank = parse(Int64, tochar(rank(square)))
    return calc_score(board, piece, score, myfile, myrank)
end 

calc_score (generic function with 2 methods)

In [44]:
b = startboard()
move = movefromstring("d3d1")
toMove = to(move)
capturePiece = get_piece(b, toMove)

calc_score(b, capturePiece, 125, toMove)

1020

In [21]:
function evaluate_position(board:: Board)
    if isterminal(board)
        return terminal_evaluation(board)
    end
    score = 0
    files = get_files()
    for x in 1:8
        for y in 1:8
            piece = get_piece(board, files[x], y)
             if piece == EMPTY
                continue
            end
            score = calc_score(board, piece, score, x, y)
        end
    end 
    return score 
end   

evaluate_position (generic function with 1 method)

In [22]:
b = startboard()
domove!(b, "e4")
domove!(b, "d5")
domove!(b, "d5")
b

Board (rnbqkbnr/ppp1pppp/8/3P4/8/8/PPPP1PPP/RNBQKBNR b KQkq -):
 r  n  b  q  k  b  n  r 
 p  p  p  -  p  p  p  p 
 -  -  -  -  -  -  -  - 
 -  -  -  P  -  -  -  - 
 -  -  -  -  -  -  -  - 
 -  -  -  -  -  -  -  - 
 P  P  P  P  -  P  P  P 
 R  N  B  Q  K  B  N  R

In [23]:
evaluate_position(b)

125

## Ohne Inkrementelle Evaluation eines Zuges

In [24]:
function evaluate_move(board, move)
    oldScore = evaluate_position(board)
    newBoard = domove(board, move)
    newScore = evaluate_position(board)
    return newScore - oldScore
end

evaluate_move (generic function with 1 method)

## Inkrementelle Evaluation eines Zuges

In [25]:
function subPiece(board, piece, toMove, score)
        score *= -1
        score = calc_score(board, piece, score, toMove)
        score *= -1
    return score
end

subPiece (generic function with 1 method)

In [26]:
epsquare(b)

SQ_NONE

In [27]:
function subCapturePiece(board, toMove, score)    
    capturePiece = get_piece(board, toMove)
    if (capturePiece != EMPTY)
        score = subPiece(board,capturePiece, toMove, score)
    end
    return score
end

subCapturePiece (generic function with 1 method)

In [28]:
move = movefromstring("d8d5")
toMove = to(move)
subCapturePiece(b,toMove, 925)

800

In [29]:
function checkEnPassant(board, toMove, score)
    enpassantSquare = epsquare(board)
    if(enpassantSquare != SQ_NONE && enpassantSquare == toMove)
        print("SubCapture")
        lastToMove = to(lastmove(board))
        score = subCapturePiece(board, lastToMove, score)
    end 
    return score
end 

checkEnPassant (generic function with 1 method)

In [30]:
checkEnPassant(b, movefromstring("c7c5"), 125)

125

In [31]:
domove!(b, "c5")
b

Board (rnbqkbnr/pp2pppp/8/2pP4/8/8/PPPP1PPP/RNBQKBNR w KQkq c6):
 r  n  b  q  k  b  n  r 
 p  p  -  -  p  p  p  p 
 -  -  -  -  -  -  -  - 
 -  -  p  P  -  -  -  - 
 -  -  -  -  -  -  -  - 
 -  -  -  -  -  -  -  - 
 P  P  P  P  -  P  P  P 
 R  N  B  Q  K  B  N  R

In [32]:
checkEnPassant(b, Square(FILE_C, RANK_6), 125)

SubCapture

225

`calc_castle` checks wether the `move` done was a castleing move. 

If no the score will be returned back.

If yes it will adds the value of the move rook. The moved King will be calculated as the regularmove in the `evaluate_move` function. Casteling Kingside does not effect the value of the rook. Casteling Queenside adds +5 to the value of the position.

In [33]:
function calc_castle(board, move, score)
    toMove = to(move)
    fromMove = from(move)
    piece = get_piece(board, fromMove)
    if(ptype(piece) == KING && file(fromMove) == FILE_E)
        # Check Castle Kingside
        if(file(toMove)==FILE_G) 
            return score
        # Check Castle Queenside
        elseif(file(toMove)==FILE_C)
            return score + 5
        end
    end 
    return score
end

calc_castle (generic function with 1 method)

In [34]:
b1 = fromfen("r3k2r/pppppppp/8/8/8/8/PPPPPPPP/R3K2R w KQkq - 0 1")
calc_castle(b1, movefromstring("e1c1"), 0)

5

In [35]:
function evaluate_move(board:: Board, move:: Move, oldscore)
    if isterminal(domove(board, move))
        return terminal_evaluation(domove(board, move))
    end
    newscore = oldscore
    toMove = to(move)
    fromMove = from(move)
    piece = get_piece(board, fromMove)
    newscore = subCapturePiece(board, toMove, newscore)
    if ptype(piece) == PAWN
        checkEnPassant(board, toMove, newscore)
    end
    # TODO: define and check for endgame state for king calculation
    newscore = subPiece(board, piece, fromMove, newscore)
    if ispromotion(move)
        piece = (sidetomove(board), promotion(move))
    end
    newscore = calc_castle(board, move, newscore)
    return calc_score(board, piece, newscore, toMove)
end

evaluate_move (generic function with 2 methods)

In [36]:
function terminal_evaluation(board:: Board)
    if ischeckmate(board)
        color = sidetomove(board)
        if color == WHITE
            return -100000
        else
            return 100000
        end
    elseif isstalemate(board)
        return 0
    elseif ismaterialdraw(board)
        return 0
    elseif isrule50draw(board)
        return 0
    end
    return
end

terminal_evaluation (generic function with 1 method)

In [37]:
move = movefromstring("f1e2")
evaluate_move(b, move,125)

135

In [38]:
b = fromfen("8/4k3/8/4p3/8/2BK4/8/q7 w - - 0 1")
display(b)
evaluate_position(b)

Board (8/4k3/8/4p3/8/2BK4/8/q7 w - -):
 -  -  -  -  -  -  -  - 
 -  -  -  -  k  -  -  - 
 -  -  -  -  -  -  -  - 
 -  -  -  -  p  -  -  - 
 -  -  -  -  -  -  -  - 
 -  -  B  K  -  -  -  - 
 -  -  -  -  -  -  -  - 
 q  -  -  -  -  -  -  -

-630

In [39]:
b = fromfen("8/4k3/8/B3p3/8/3K4/8/q7 w - - 0 1")
evaluate_position(b)

-650

In [40]:
b = fromfen("8/4k3/8/4p3/8/2BK4/8/q7 w - - 0 1")
evaluate_move(b, movefromstring("c3a5"), -680)

-700

In [41]:
b = fromfen("k7/4Q3/2K5/8/8/8/8/8 w - - 3 3")
evaluate_move(b, movefromstring("e7b7"), 0)
b

Board (k7/4Q3/2K5/8/8/8/8/8 w - -):
 k  -  -  -  -  -  -  - 
 -  -  -  -  Q  -  -  - 
 -  -  K  -  -  -  -  - 
 -  -  -  -  -  -  -  - 
 -  -  -  -  -  -  -  - 
 -  -  -  -  -  -  -  - 
 -  -  -  -  -  -  -  - 
 -  -  -  -  -  -  -  -